In [1]:
from __future__ import division
from pyomo.environ import *
import numpy as np
import pandas as pd
from folium import Map, CircleMarker, FeatureGroup, LayerControl
import os.path
import shapely.wkt
from src.distance_calc_utils import haversine_distance_matrix

## Calculate distances

In [2]:
# Import demand
demand = pd.read_csv('../data/interim/lp_data/input_data/Demand_Model_Output.csv')
demand = demand[demand.A > 0]
demand['geometry'] = demand.geometry.apply(lambda x: shapely.wkt.loads(x))
demand['longitude'], demand['latitude'] =  demand.geometry.apply(lambda x: x.centroid.x), demand.geometry.apply(lambda x: x.centroid.y)

In [4]:
# Calculate and filter demand
distances = haversine_distance_matrix(demand)
distances = distances[distances['dist'] <= 10]

## Create Sets

In [15]:
model = AbstractModel()

In [16]:
node_list = list(demand.B)
charger_list = [1,2]
time_list = list(range(1,25))
line_list = list(distances.index)

# Create pyomo sets
model.B = Set(initialize=node_list)
model.K = Set(initialize=charger_list)
model.T = Set(initialize=time_list)
model.L = Set(initialize=line_list)

## Create Parameters from Sets

In [17]:
# Create Model Parameters
model.F = Param(model.B, model.K)
model.D = Param(model.B, model.K)
model.p = Param(model.B, model.L)
model.A = Param(model.B, model.T)
model.G = Param(model.T)
model.C = Param(model.B, model.K)
model.N = Param(model.K)
model.E = Param(model.B, model.K)
model.S = Param(model.B)
model.M = Param(initialize=100)
model.VW = Param(model.B, model.K, model.T)
model.P_H_U = Param(model.L, model.T)

## Generate Parameter Files

### Generate Fixed Costs DataFrame

In [51]:
fixed_costs = pd.DataFrame(index=node_list, columns=['K', 'F'])
charger_costs = dict(zip(charger_list, [320, 365]))

In [56]:
fixed_cost_final = []

for i in charger_list:
    fixed_costs_copy = fixed_costs.copy()
    fixed_costs_copy['K'] = i
    fixed_costs_copy['F'] = charger_costs[i]
    fixed_cost_final.append(fixed_costs_copy)

fixed_costs = pd.concat(fixed_cost_final)

In [60]:
fixed_costs = fixed_costs.reset_index().rename(columns={'index':'B'})

### Generate Demand Charge

In [71]:
demand_charges = pd.DataFrame(index=node_list, columns=['K', 'D'])

In [72]:
demand_charges_final = []

for i in charger_list:
    demand_charges_copy = demand_charges.copy()
    demand_charges_copy['K'] = i
    demand_charges_copy['D'] = 0
    demand_charges_final.append(demand_charges_copy)

demand_charges = pd.concat(demand_charges_final)

In [74]:
demand_charges = demand_charges.reset_index().rename(columns={'index':'B'})

### Generate Incidence Matrix

In [82]:
all_lines = []
for id1 in node_list:
    for id2 in node_list:
        all_lines.append(f"{id1}_{id2}")

In [83]:
# Because distance calculation both directions are the same
incidence_matrix = pd.DataFrame(index=node_list, columns=all_lines)

In [99]:
for index, row in incidence_matrix.iterrows():
    for line in row.index:
        if (index in line.split('_')) and (index == line.split('_')[0]):
            row[line] = 1
        elif index == line.split('_')[1]:
            row[line] = -1
        else:
            row[line] = 0

In [102]:
# Need to do in and 1,-1
incidence_matrix.loc['8829a19903fffff']

,8829a19907fffff_8829a19907fffff,8829a19907fffff_8829a19903fffff,8829a19907fffff_8829a0b2d5fffff,8829a19907fffff_8829a0b553fffff,8829a19907fffff_8829a56a5dfffff,8829a19907fffff_8829a18a97fffff,8829a19907fffff_8829a18ae5fffff,8829a19907fffff_8829a1983dfffff,8829a19907fffff_8829a0adebfffff,8829a19907fffff_8829a57b2dfffff,...,8829a0b2d5fffff_8829a18917fffff,8829a0b2d5fffff_8829a0aee3fffff,8829a0b2d5fffff_8829a56869fffff,8829a0b2d5fffff_8829a1d69dfffff,8829a0b2d5fffff_8829a18807fffff,8829a0b2d5fffff_8829a56c9dfffff,8829a0b2d5fffff_8829a56869fffff,8829a0b2d5fffff_8829a09635fffff,8829a0b2d5fffff_8829a1d6b9fffff,8829a0b2d5fffff_8829a0b2d5fffff
8829a19903fffff,0,-1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8829a19903fffff,0,-1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8829a19903fffff,0,-1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Run the model

In [ ]:
from src.lp_model import linear_program
lp = linear_program()
lp.run()